In [13]:
import nltk
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline
from google.colab import files
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score


In [14]:
uploaded = files.upload()
data = pd.read_csv('dataset01.txt', sep = '\n')
print(data.head())

Saving dataset01.txt to dataset01 (1).txt
                                            all_data
0                                     >iTELL-Chr1_61
1  CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
2                                                  0
3                                   >iYAL067W-A_5085
4  AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...


In [15]:
all=data["all_data"].tolist()
cls=[]
sequence=[]
label=[]

In [16]:
length = len(all)
for i in range(length):
   if (i%3)==0:
             cls.append(all[i])
   else:
      if(all[i]== "0" or all[i]=="1"):
        label.append(all[i])
      else:
         sequence.append(all[i])

print(len(cls))
print(len(label))
print(len(sequence))

14965
14965
14965


In [17]:
df = pd.DataFrame(list(zip(cls, label, sequence)),
               columns =['class' , 'label', 'sequence'])
df.head(10)

,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...
5,>YAL067C_SEO1_8340,1,CACAATAGCGTCAATAATAAAGTTCCATCTCCATCCCTCTAAACCA...
6,>iYAL067C_9509,1,GTTTTTTTCCAGTGACACAATCTTTACCATTACACAGTTTTTACTA...
7,>iYAL067C_10442,0,CAGAAACATAAAATTATATCACTTTATTTCATATGGTTTCATGCTT...
8,>YAL065C_YAL065C_11570,1,AGATACCTCGAAACAAACTCTATGTAAACACTTATTTTATTGTGGT...
9,>iYAL064W-B_13286,1,ATGACTGGAGAAGAAGTGACTAGAGAAGAAGTGACTGAGGAAGAAA...


In [18]:
X = df['sequence']
Y = df['label']

In [19]:
def getKmers(sequence, size=4):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
    

In [20]:
df['words'] = df.apply(lambda x: getKmers(x['sequence']), axis=1)
df = df.drop('sequence', axis=1)
df.head(10)             

,class,label,words
0,>iTELL-Chr1_61,0,"[caca, acac, cacc, acca, ccac, caca, acac, cac..."
1,>iYAL067W-A_5085,0,"[aatt, attt, tttt, tttt, ttta, ttat, tata, ata..."
2,>iYAL067W-A_6786,0,"[aatt, atta, ttat, tata, atat, tatt, attt, ttt..."
3,>iYAL067W-A_7060,0,"[aaca, acaa, caat, aata, atag, tagt, agtg, gtg..."
4,>YAL067C_SEO1_8061,1,"[caaa, aaag, aaga, agat, gatt, attt, tttc, ttc..."
5,>YAL067C_SEO1_8340,1,"[caca, acaa, caat, aata, atag, tagc, agcg, gcg..."
6,>iYAL067C_9509,1,"[gttt, tttt, tttt, tttt, tttt, tttc, ttcc, tcc..."
7,>iYAL067C_10442,0,"[caga, agaa, gaaa, aaac, aaca, acat, cata, ata..."
8,>YAL065C_YAL065C_11570,1,"[agat, gata, atac, tacc, acct, cctc, ctcg, tcg..."
9,>iYAL064W-B_13286,1,"[atga, tgac, gact, actg, ctgg, tgga, ggag, gag..."


In [21]:
dna_texts = list(df['words'])
for item in range(len(dna_texts)):
    dna_texts[item] = ' '.join(dna_texts[item])
y_data = df['label'].values

Bag of words

In [22]:
cv = CountVectorizer(ngram_range=(4,4))
X = cv.fit_transform(dna_texts)
print(X.shape)

(14965, 16384)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y_data, test_size = 0.25,  shuffle = True) 

print(X_train.shape)
print(X_test.shape)                                         

(11223, 16384)
(3742, 16384)


In [24]:
scoring = 'accuracy'

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'AddaBoost', 'Multinomial NB',
         'SVM Linear', 'Logistic Regression']
Classifiers = [
    KNeighborsClassifier(n_neighbors = 3),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators = 100, max_features = 1 ),
    AdaBoostClassifier(),
    MultinomialNB(alpha=0.1),
    svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'),
    LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=1000)
     ]

models = zip(names, Classifiers)
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))


K Nearest Neighbors
[[1664  195]
 [ 537 1346]]
              precision    recall  f1-score   support

           0       0.76      0.90      0.82      1859
           1       0.87      0.71      0.79      1883

    accuracy                           0.80      3742
   macro avg       0.81      0.80      0.80      3742
weighted avg       0.82      0.80      0.80      3742

0.8043826830571886
Decision Tree
[[1252  607]
 [ 605 1278]]
              precision    recall  f1-score   support

           0       0.67      0.67      0.67      1859
           1       0.68      0.68      0.68      1883

    accuracy                           0.68      3742
   macro avg       0.68      0.68      0.68      3742
weighted avg       0.68      0.68      0.68      3742

0.6761090326028861
Random Forest
[[1490  369]
 [ 445 1438]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.79      1859
           1       0.80      0.76      0.78      1883

    accuracy   